In [ ]:
%run Lib_full_vs_partial_charge.ipynb
import operator
from datetime import datetime
from datetime import timedelta
import csv
from csv import writer
from csv import DictWriter
import database_io as db
import reverse_geocoder as rg 
import threading

In [ ]:
def compute_dif_of_date(date1,date2):
    first_date = datetime.strptime(date1, "%Y-%m-%d %H:%M:%S")
    second_date= datetime.strptime(date2, "%Y-%m-%d %H:%M:%S")
    date1 = datetime(first_date.year, first_date.month, first_date.day, first_date.hour, first_date.minute, first_date.second)
    date2 = datetime(second_date.year, second_date.month, second_date.day, second_date.hour, second_date.minute, second_date.second)
    diff = date1-date2
    #print(diff)
    #print(diff.days) 
    #print(diff.seconds) just shows the seconds for hours not include day
    return diff

In [ ]:
def compute_date(date,time_in_second):
    date = datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
    date = datetime(date.year, date.month, date.day, date.hour, date.minute, date.second)
    a = datetime(date.year, date.month, date.day, date.hour, date.minute, date.second)
    final_date = a + timedelta(0,time_in_second) # days, seconds, then other fields.
    final_date=str(final_date)
    final_date=final_date[:len(final_date)-7]

    return final_date

In [ ]:
def finding_consumption_between_nodes(G,Gsub_D,start_point_osm_id,end_point_osm_id,main_arrival_battery):
    ss=0
    cons_for_node_no_need_charge=0
    percorso_tempo_batt_eur3=euristica3Knn(G,Gsub_D,[start_point_osm_id,end_point_osm_id],main_arrival_battery)
    len_list=len(percorso_tempo_batt_eur3[(start_point_osm_id,end_point_osm_id)])
    list_of_stops=percorso_tempo_batt_eur3[(start_point_osm_id,end_point_osm_id)][len_list-2]
    for node_ids in list_of_stops:
        if ss >= len(list_of_stops)-1:
            break
        if 'consumption' in G[list_of_stops[ss]][list_of_stops[ss+1]][0]:
            cons_for_node_no_need_charge+=float(G[list_of_stops[ss]][list_of_stops[ss+1]][0]['consumption']) 
        ss+=1
    cons_for_node_no_need_charge=(cons_for_node_no_need_charge/3600*1e3)*1e-6
    return cons_for_node_no_need_charge

In [ ]:
def charge_level_station(charge_node):
    if 'Level 1' in charge_node:
        charge_level=1
    if 'Level 2' in charge_node:
        charge_level=2
    if 'Level 3' in charge_node:
        charge_level=3
    return charge_level

In [ ]:
def reverseGeocode(coordinates): 
    result = rg.search(coordinates) 
    tt=0
    for val in result[0].values():
        if tt==3:
            if val=='Tuscany':
                return True
            else:
                return False
        tt+=1

In [ ]:
def find_home_work_osm_ids(user):
    try:
        kk=0
        for key,value in user['location_prototype'].items():
            if int(key) in user['regular_locations']:
                if kk==0:
                    home=(user['location_prototype'][key][1],user['location_prototype'][key][0])
                if kk==1:
                    work=(user['location_prototype'][key][1],user['location_prototype'][key][0])
                kk+=1
    except(TypeError):
        home=None
        work=None
    return home,work

In [ ]:
def append_list_as_row(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

def append_dict_as_row(file_name, dict_of_elem, field_names):
    with open(file_name, 'a+', newline='') as write_obj:
        dict_writer = DictWriter(write_obj, fieldnames=field_names)
        dict_writer.writerow(dict_of_elem)

In [ ]:
def charge_full_at_location(user_id,main_arrival_battery,battery_capacity,location,end_time_h,field_names,file_csv_name):
    arrival_battery_amount=main_arrival_battery
    battery_amount=battery_capacity-main_arrival_battery
    #print("amonut of battery which we add in emergency call is : ",battery_amount)
    waiting_time=battery_amount*6

    row_dict={'user_id': user_id, 'charge_location': location,'charge_level': 2,'battery_level_when_arrive_to_station':round(arrival_battery_amount,2),'amount_battery_charge':round(battery_amount,2),'waiting_time':round(waiting_time,2),'arrive_time':end_time_h}
    append_dict_as_row(file_csv_name, row_dict, field_names)
    main_arrival_battery=battery_capacity    
    
    return main_arrival_battery

In [ ]:
def charging_at_home_work_tuscany_sample_user(battery,file_csv_name,stay_time=240,home_flag=True,work_flag=True,battery_capacity=40):
    # TODO: the function assumes a specific format of data. There are several IMNs, 
    # one for each time period: "01-02", etc. Please adapt the code to your needs.
    
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')  
    k=0
    m=0
    end_point_osm_id=0
    start_point_osm_id=0
    emergency_list=list()
    home=tuple()
    work=tuple()  
    field_names = ['user_id','charge_location','charge_level','battery_level_when_arrive_to_station','amount_battery_charge','waiting_time','arrive_time']
    for row in data:
        m=0
        k=0
        user_data = json.loads(row)
        user_id=user_data['uid']
        if user_id==973:
            try:
                user=user_data['01-02']
            except(KeyError):
                print("key error")
                continue
            print("user ",user_id," is start")
            home,work=find_home_work_osm_ids(user)
            if home is None or work is None:
                continue
            if not reverseGeocode(home):
                print("user is not in tuscany")
                continue
            home_osm_id = ox.get_nearest_node(G, home)                
            work_osm_id = ox.get_nearest_node(G, work)
            if home_osm_id==work_osm_id:
                print("home and work point are same")
                work_flag=False
                home_flag=True
            sorted_dict_list = sorted(user['movement_prototype'].values(), key = lambda x:x['_start_time'], reverse=False)    
            for main_dict in sorted_dict_list:
                k=0
                mov_obj=main_dict['object']
                len_mov=len(mov_obj)
                for mov in mov_obj:
                    if k==0:
                        start_point=(mov[1],mov[0])
                        start_point_osm_id = ox.get_nearest_node(G, start_point)
                    if k==len_mov-1:
                        end_point=(mov[1],mov[0])
                        end_point_osm_id = ox.get_nearest_node(G, end_point)
                    k+=1
                if m==0:
                    main_arrival_battery=battery
                if start_point_osm_id!=end_point_osm_id:
                    if m < len(sorted_dict_list)-1 :
                        start_time_h=sorted_dict_list[m+1]['_start_time']
                        end_time_h=sorted_dict_list[m]['_end_time']
                        try:
                            dic,main_info_list_full,main_arrival_battery=charging_info_at_charging_station(start_point_osm_id,end_point_osm_id,main_arrival_battery,0,battery_capacity,True,False)  
                            if not dic and main_arrival_battery==battery_capacity:
                                cons_for_node_no_need_charge=finding_consumption_between_nodes(G,Gsub_D,start_point_osm_id,end_point_osm_id,main_arrival_battery)
                                #print("consumption for the trip which there is no need to charge",cons_for_node_no_need_charge)
                                main_arrival_battery=main_arrival_battery-cons_for_node_no_need_charge
                            if main_info_list_full:
                                for main_info_list in main_info_list_full:
                                    time_in_sec=main_info_list[3]
                                    time_arrive_station = compute_date(end_time_h,time_in_sec)
                                    arr_battery_amount=round(battery_capacity-main_info_list[1],2)
                                    charge_level=charge_level_station(G.nodes[main_info_list[0]]['conInf'])
                                    row_dict={'user_id': user_id, 'charge_location': 'public','charge_level': charge_level,'battery_level_when_arrive_to_station':arr_battery_amount,'amount_battery_charge':main_info_list[1],'waiting_time':main_info_list[2],'arrive_time':time_arrive_station}
                                    append_dict_as_row(file_csv_name, row_dict, field_names)

                            #print("the intial battery for the next trip",main_arrival_battery)
                            if home_flag:
                                if end_point_osm_id==home_osm_id:
                                    diff=compute_dif_of_date(start_time_h,end_time_h)
                                    if (diff.days >-1) or (diff.seconds/60>stay_time):
                                            #print("we arrive at home at :",end_time_h,"we start from home at :",start_time_h)
                                            main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'home',end_time_h,field_names,file_csv_name)
                            if work_flag:    
                                if end_point_osm_id==work_osm_id:
                                    diff=compute_dif_of_date(start_time_h,end_time_h)
                                    if (diff.days >-1) or (diff.seconds/60>stay_time):
                                        try:
                                            #print("we arrive at work at :",end_time_h,"we start from work at :",start_time_h)
                                            main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'work',end_time_h,field_names,file_csv_name)
                                        except(TypeError):
                                            continue
                        except(TypeError):
                            #print("emergnecy")
                            if main_arrival_battery<0:
                                continue
                            main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'emergency',end_time_h,field_names,file_csv_name)
                            continue   
                        except(KeyError):
                            print("key error")
                            continue 
                m+=1 

### MAIN ALGORITHM WITH EMERGENCY

In [ ]:
def charging_at_home_work_tuscany(battery,file_csv_name,stay_time=240,home_flag=True,work_flag=True,battery_capacity=40):
    # TODO: the function assumes a specific format of data. There are several IMNs, 
    # one for each time period: "01-02", etc. Please adapt the code to your needs.
    
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')  
    k=0
    m=0
    end_point_osm_id=0
    start_point_osm_id=0
    emergency_list=list()
    home=tuple()
    work=tuple()  
    field_names = ['user_id','charge_location','charge_level','battery_level_when_arrive_to_station','amount_battery_charge','waiting_time','arrive_time']
    for row in data:
        m=0
        k=0
        user_data = json.loads(row)
        user_id=user_data['uid']
        try:
            user=user_data['01-02']
        except(KeyError):
            print("key error")
            continue
        print("user ",user_id," is start")
        home,work=find_home_work_osm_ids(user)
        if home is None or work is None:
            continue
        if not reverseGeocode(home):
            print("user is not in tuscany")
            continue
        home_osm_id = ox.get_nearest_node(G, home)                
        work_osm_id = ox.get_nearest_node(G, work)
        print("home",home_osm_id,"work",work_osm_id)
        if home_osm_id==work_osm_id:
            print("home and work point are same")
            #work_flag=False
            #home_flag=True
        sorted_dict_list = sorted(user['movement_prototype'].values(), key = lambda x:x['_start_time'], reverse=False)    
        for main_dict in sorted_dict_list:
            k=0
            mov_obj=main_dict['object']
            len_mov=len(mov_obj)
            for mov in mov_obj:
                if k==0:
                    start_point=(mov[1],mov[0])
                    start_point_osm_id = ox.get_nearest_node(G, start_point)
                if k==len_mov-1:
                    end_point=(mov[1],mov[0])
                    end_point_osm_id = ox.get_nearest_node(G, end_point)
                k+=1
            if m==0:
                main_arrival_battery=battery
            if start_point_osm_id!=end_point_osm_id:
                if m < len(sorted_dict_list)-1 :
                    start_time_h=sorted_dict_list[m+1]['_start_time']
                    end_time_h=sorted_dict_list[m]['_end_time']
                    try:
                        dic,main_info_list_full,main_arrival_battery=charging_info_at_charging_station(start_point_osm_id,end_point_osm_id,main_arrival_battery,0,battery_capacity,True,False)  
                        if not dic and main_arrival_battery==battery_capacity:
                            cons_for_node_no_need_charge=finding_consumption_between_nodes(G,Gsub_D,start_point_osm_id,end_point_osm_id,main_arrival_battery)
                            #print("consumption for the trip which there is no need to charge",cons_for_node_no_need_charge)
                            main_arrival_battery=main_arrival_battery-cons_for_node_no_need_charge
                        if main_info_list_full:
                            for main_info_list in main_info_list_full:
                                time_in_sec=main_info_list[3]
                                time_arrive_station = compute_date(end_time_h,time_in_sec)
                                arr_battery_amount=round(battery_capacity-main_info_list[1],2)
                                charge_level=charge_level_station(G.nodes[main_info_list[0]]['conInf'])
                                row_dict={'user_id': user_id, 'charge_location': 'public','charge_level': charge_level,'battery_level_when_arrive_to_station':arr_battery_amount,'amount_battery_charge':main_info_list[1],'waiting_time':main_info_list[2],'arrive_time':time_arrive_station}
                                append_dict_as_row(file_csv_name, row_dict, field_names)
                        
                        #print("the intial battery for the next trip",main_arrival_battery)
                            print("we are in public and start node is ",start_point_osm_id," end node",end_point_osm_id,"the inital battery for the next trip is ",main_arrival_battery)
                        if home_flag:
                            if end_point_osm_id==home_osm_id:
                                diff=compute_dif_of_date(start_time_h,end_time_h)
                                if (diff.days >-1) or (diff.seconds/60>stay_time):
                                        #print("we arrive at home at :",end_time_h,"we start from home at :",start_time_h)
                                        main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'home',end_time_h,field_names,file_csv_name)
                                print("we are in home and start node is ",start_point_osm_id," end node",end_point_osm_id,"the inital battery for next trip is ",main_arrival_battery)
                        if work_flag:    
                            if end_point_osm_id==work_osm_id:
                                diff=compute_dif_of_date(start_time_h,end_time_h)
                                if (diff.days >-1) or (diff.seconds/60>stay_time):
                                    try:
                                        #print("we arrive at work at :",end_time_h,"we start from work at :",start_time_h)
                                        main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'work',end_time_h,field_names,file_csv_name)
                                        print("we are in work and start node is ",start_point_osm_id," end node",end_point_osm_id,"the inital battery for next trip is ",main_arrival_battery)
                                        
                                    except(TypeError):
                                        continue
                    except(TypeError):
                        #print("emergnecy")
                        if main_arrival_battery<0:
                            continue
                        main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'emergency',end_time_h,field_names,file_csv_name)
                        print("we are in emergency and start node is ",start_point_osm_id," end node",end_point_osm_id,"the inital battery for next trip is ",main_arrival_battery)
                        continue   
                    except(KeyError):
                        print("key error")
                        continue 
            m+=1 

In [ ]:
#type charge 1 is slow 2 is medium and 3 is fast
def time_charge_model_per_kw(value,battery_need):
    min_wait=0
    if '2' in value:
        #it is fast charge which is 4 hours for filling 40kw so it is 240 min and 6 min per kw
        min_wait=6
        
    elif '1' in value:
        #it is slow charge which is 6 hours for filling 40kw so it is 360 min and 9 min per kw
        min_wait=9
        
    elif '3' in value:
        #it is fast charge which charge 80% in 30 min so 1.06 per kw from 0 to 32kw and from 32 to 40 1.5 min 
        if battery_need < 32:
            min_wait=1.06
        else :
            min_wait=1.5
        
    return min_wait 

### MAIN ALGORITHM WITH SOLVING EMERGENCY

In [ ]:
def charging_at_home_work_tuscany_solve_emergency(battery,file_csv_name,stay_time=240,home_flag=True,work_flag=True,battery_capacity=40):
    # TODO: the function assumes a specific format of data. There are several IMNs, 
    # one for each time period: "01-02", etc. Please adapt the code to your needs.
    
    input_filename = 'GZIPPED-JSON-FILE-WITH-IMNS' # TODO: replace with file name
    data = gzip.GzipFile(input_filename, 'r')  
    k=0
    m=0
    end_point_osm_id=0
    start_point_osm_id=0
    emergency_list=list()
    home=tuple()
    work=tuple()  
    start_end_batt=dict()
    initial_battery_of_last_trip=0
    emergency_flag=False
    hidden_batt=list()
    start_batt=0
    end_batt=0
    info_last_trip=list()
    key_flag=False
    list_m=[]
    first_emergency=False
    unavoidable_emergency=False
    amount_battery_charge=0
    waiting_time=0
    field_names = ['user_id','charge_location','charge_level','battery_level_when_arrive_to_station','amount_battery_charge','waiting_time','arrive_time']
    for row in data:
        main_arrival_battery=0
        new_batt=0
        start_end_batt={}
        list_m=[]
        hidden_batt=[]
        m=0
        k=0
        user_data = json.loads(row)
        user_id=user_data['uid']
        if user_id == 954:
            continue
        try:
            user=user_data['01-02']
        except(KeyError):
            print("key error")
            continue
        print("user ",user_id," is start")
        home,work=find_home_work_osm_ids(user)
        if home is None or work is None:
            continue
        if not reverseGeocode(home):
            print("user is not in tuscany")
            continue
        home_osm_id = ox.get_nearest_node(G, home)                
        work_osm_id = ox.get_nearest_node(G, work)
        print("home",home_osm_id,"work",work_osm_id)
        if home_osm_id==work_osm_id:
            print("home and work point are same")
            continue
        sorted_dict_list = sorted(user['movement_prototype'].values(), key = lambda x:x['_start_time'], reverse=False)    
        #for m in range(len(sorted_dict_list)-1):
        while m < len(sorted_dict_list)-1:
            if key_flag:
                key_flag=False 
                m+=1
                continue
            if emergency_flag:  
                m-=1
                if m<=0 :
                    battery=battery+5
                else:
                    m-=1
                    try:
                        info_last_trip=start_end_batt[m]
                    except(KeyError):
                        #print("key error vaseye etelaaat last trip hala print mikonim start va end batt",start_end_batt, " m",m)
                        info_last_trip=start_end_batt[m-1]
                        m=m-1
                    #print("start o end ghable",info_last_trip)
                    if info_last_trip[0]==battery_capacity:
                        m=max(list_m)-1
                        new_batt=charge_at_special_location(user_id,start_end_batt[m-1][1],battery_capacity,'emergency',end_time_h,field_names,file_csv_name)
                        #print("unavoidable",new_batt)
                    else:
                        batt_need_to_arrive=info_last_trip[0]-info_last_trip[1]
                        new_batt=batt_need_to_arrive-((batt_need_to_arrive*5)/100)
                        if new_batt <=0:
                            new_batt=40-batt_need_to_arrive+new_batt
                    hidden_batt.append(info_last_trip[0]- new_batt)
            if new_batt!=0:
                first_emergency=True
                main_arrival_battery=new_batt
                emergency_flag=False
            #print("main battery bad az inke az emergency kharej mishe",main_arrival_battery)
            k=0
            mov_obj=sorted_dict_list[m]['object']
            len_mov=len(mov_obj)
            for mov in mov_obj:
                if k==0:
                    start_point=(mov[1],mov[0])
                    start_point_osm_id = ox.get_nearest_node(G, start_point)
                if k==len_mov-1:
                    end_point=(mov[1],mov[0])
                    end_point_osm_id = ox.get_nearest_node(G, end_point)
                k+=1
            if m==0:
                if first_emergency:
                    main_arrival_battery=40
                    first_emergency=False
                else:
                    main_arrival_battery=battery
            if start_point_osm_id!=end_point_osm_id:
                if m < len(sorted_dict_list)-1 :
                    start_time_h=sorted_dict_list[m+1]['_start_time']
                    end_time_h=sorted_dict_list[m]['_end_time']
                    try:
                        print("we start with",main_arrival_battery," with ",start_point_osm_id,end_point_osm_id)
                        start_batt=main_arrival_battery
                        dic,main_info_list_full,main_arrival_battery=charging_info_at_charging_station(start_point_osm_id,end_point_osm_id,start_batt,0,battery_capacity,True,False)  
                        if not dic and main_arrival_battery==battery_capacity:
                            cons_for_node_no_need_charge=finding_consumption_between_nodes(G,Gsub_D,start_point_osm_id,end_point_osm_id,main_arrival_battery)
                            #print("consumption for the trip which there is no need to charge",cons_for_node_no_need_charge)
                            main_arrival_battery=start_batt-cons_for_node_no_need_charge
                            if main_arrival_battery<=0:
                                main_arrival_battery=40-cons_for_node_no_need_charge+start_batt

                        if main_info_list_full:
                            #print("we need to charge")
                            for main_info_list in main_info_list_full:
                                time_in_sec=main_info_list[3]
                                time_arrive_station = compute_date(end_time_h,time_in_sec)
                                main_arrival_battery=round(battery_capacity-main_info_list[1],2)
                                charge_level=charge_level_station(G.nodes[main_info_list[0]]['conInf'])
                                if hidden_batt:
                                    #print("public battery mogheyi ke miresom ghabl add hidden",main_arrival_battery)
                                    main_arrival_battery=main_arrival_battery+hidden_batt[len(hidden_batt)-1]
                                    #print("public battery mogheyi ke miresom bade add hidden",main_arrival_battery)
                                    if main_arrival_battery > 40 :
                                            main_arrival_battery=40
                                    
                                    #hidden_batt=[]
                                    amount_battery_charge=round((40-main_arrival_battery),2)
                                    waiting_time=time_charge_model_per_kw(str(charge_level),amount_battery_charge)
                                    waiting_time=round((waiting_time*amount_battery_charge),2)
                                else:
                                    amount_battery_charge=main_info_list[1]
                                    waiting_time=main_info_list[2]
                                    
                                row_dict={'user_id': user_id, 'charge_location': 'public','charge_level': charge_level,'battery_level_when_arrive_to_station':round(main_arrival_battery,2),'amount_battery_charge':amount_battery_charge,'waiting_time':waiting_time,'arrive_time':time_arrive_station}
                                append_dict_as_row(file_csv_name, row_dict, field_names)
                                main_arrival_battery=battery_capacity
                        if home_flag:
                            if end_point_osm_id==home_osm_id:
                                diff=compute_dif_of_date(start_time_h,end_time_h)
                                if (diff.days >-1) or (diff.seconds/60>stay_time) or (main_arrival_battery!=40.0):
                                    if hidden_batt:
                                        #print("home battery mogheyi ke miresom ghable add hidden",main_arrival_battery)
                                        main_arrival_battery=main_arrival_battery+hidden_batt[len(hidden_batt)-1]
                                        #print("home battery mogheyi ke miresom bade add hidden",main_arrival_battery)
                                        if main_arrival_battery > 40 :
                                            main_arrival_battery=40
                                        #hidden_batt=[]
                                    main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'home',end_time_h,field_names,file_csv_name)
                                    #print("home charge")
                        if work_flag:    
                            if end_point_osm_id==work_osm_id:
                                diff=compute_dif_of_date(start_time_h,end_time_h)
                                if (diff.days >-1) or (diff.seconds/60>stay_time) or (main_arrival_battery!=40.0):
                                    try:
                                        if hidden_batt:
                                            #print("work battery mogheyi ke miresom ghable add hidden",main_arrival_battery)
                                            main_arrival_battery=main_arrival_battery+hidden_batt[len(hidden_batt)-1]
                                            #print("work battery mogheyi ke miresom bade add hidden",main_arrival_battery)
                                            if main_arrival_battery > 40 :
                                                main_arrival_battery=40
                                            #hidden_batt=[]
                                        main_arrival_battery=charge_full_at_location(user_id,main_arrival_battery,battery_capacity,'work',end_time_h,field_names,file_csv_name)
                                        #print("work charge")
                                    except(TypeError):
                                        continue
                        end_batt=main_arrival_battery
                        if m in start_end_batt:
                            del start_end_batt[m]
                        start_end_batt[m]=[start_batt,end_batt]
                        #print("we arrive with ",main_arrival_battery)
                        new_batt=main_arrival_battery
                    except(TypeError):
                        #print("emergency")
                        emergency_flag=True
                    except(KeyError):
                        key_flag=True
                        #print("key error")
                        continue 
            m+=1
            if m not in list_m:
                list_m.append(m)
        

In [ ]:
charging_at_home_work_tuscany_solve_emergency(5,"test.csv",stay_time=240,home_flag=False,work_flag=False,battery_capacity=40)

In [ ]:
import csv
main_list=list()
edit_row=False
counter_list=list()
field_names = ['user_id','charge_location','charge_level','battery_level_when_arrive_to_station','amount_battery_charge','waiting_time','arrive_time']
append_list_as_row('home_work_new.csv', field_names)
with open('home_work.csv', newline='') as f:
    reader = csv.reader(f)

    for row in reader:
        main_list.append(row)
    
    for i in range(len(main_list)-1):
        if i>4:
            #if main_list[i][0]=='1003':
            if (main_list[i][1]=='public' and main_list[i+1][1]=='home' and main_list[i][0]==main_list[i+1][0]) or (main_list[i][1]=='public' and main_list[i+1][1]=='work' and main_list[i][0]==main_list[i+1][0]):
            #if (main_list[i][1]=='public' and main_list[i+1][1]=='work' and main_list[i][0]==main_list[i+1][0]):

                #if (main_list[i+1][1]=='home' and main_list[i][0]==main_list[i+1][0]) or (main_list[i+1][1]=='work' and main_list[i][0]==main_list[i+1][0]):
                if i not in counter_list:
                    if float(main_list[i+1][4]) < float(main_list[i][3]):
                        amount_battery_charge1=1.0
                        battery_level_when_arrive_to_station=float(main_list[i][3])+1-float(main_list[i+1][4])
                        battery_level_when_arrive_to_station=round(battery_level_when_arrive_to_station,2)
                    else :
                        amount_battery_charge1=float(main_list[i+1][4])- float(main_list[i][3])+1
                        amount_battery_charge1=round(amount_battery_charge1,2)
                        battery_level_when_arrive_to_station=1.0

                    waiting_time1=time_charge_model_per_kw(main_list[i][2],amount_battery_charge1)
                    waiting_time1=round((waiting_time1*amount_battery_charge1),2)
                    row_dict1={'user_id': main_list[i][0], 'charge_location':  main_list[i][1],'charge_level':  main_list[i][2],'battery_level_when_arrive_to_station': main_list[i][3],'amount_battery_charge':amount_battery_charge1,'waiting_time': waiting_time1,'arrive_time': main_list[i][6]}
                    append_dict_as_row("home_work_new.csv", row_dict1, field_names)

                    #battery_level_when_arrive_to_station=float(main_list[i][3])
                    amount_battery_charge2=round(40-battery_level_when_arrive_to_station)
                    #print(amount_battery_charge2)
                    waiting_time2=time_charge_model_per_kw(main_list[i+1][2],amount_battery_charge2)
                    waiting_time2=round((waiting_time2*amount_battery_charge2),2)
                    if main_list[i+1][4] is '0':
                        battery_level_when_arrive_to_station=amount_battery_charge1+float(main_list[i][3])

                    row_dict2={'user_id': main_list[i+1][0], 'charge_location':  main_list[i+1][1],'charge_level':  main_list[i+1][2],'battery_level_when_arrive_to_station': battery_level_when_arrive_to_station,'amount_battery_charge':amount_battery_charge2,'waiting_time': waiting_time2,'arrive_time': main_list[i+1][6]}
                    append_dict_as_row("home_work_new.csv", row_dict2, field_names)
                    counter_list.append(i+1)
                    row_dict1={}
                    row_dict2={}
            else:
                if i not in counter_list:
                    row_dict3={'user_id': main_list[i][0], 'charge_location':  main_list[i][1],'charge_level':  main_list[i][2],'battery_level_when_arrive_to_station': main_list[i][3],'amount_battery_charge': main_list[i][4],'waiting_time': main_list[i][5],'arrive_time': main_list[i][6]}
                    append_dict_as_row("home_work_new.csv", row_dict3, field_names)
                    row_dict3={}